<div class="alert alert-info">
<h2> Комментарий студента_V0 <a class="tocSkip"> </h2>

Привет!<br>
Я Дмитрий и можем сразу на ты ;)<br>
</div>

<div class="alert alert-info">
<h2> Комментарий студента_V0 <a class="tocSkip"> </h2>

Решил ворваться сразу с ноги и поделится своей болью...<br>
Изначально я пробовал сделать с BERT - дня 3 возился так и не смог преодолеть порог в 0.745.......................(и далее непередаваемая игра слов)<br>
Тут собстна несколько вопросов:<br>
1) Я много вариантов перепробовал и ручные эмбединги и через готовые модели, но кроме скорости ничего не получилось - тут вообще реально сделать 0,75?<br>
2) Если реальано (ну что-то мне подсказывает что да), то может подскажешь однозначный метод (без фанатизма, может просто есть ультимативный инструмент) через который возможно достигнуть результата по ТЗ?<br>
3) TF-IDF - получился с 1го раза, если это можно так назвать, поэтому дальше о нем...
</div>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорты

In [1]:
# Стандартная библиотека
import logging
import warnings
import re
import os

# Научные и аналитические библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier

# Бустинг (используется в коде, но сейчас выключен)
from lightgbm import LGBMClassifier

# Работа с текстом
from sklearn.feature_extraction.text import TfidfVectorizer

# Оптимизация гиперпараметров
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Прогресс
from tqdm import tqdm
tqdm.pandas()

# Логирование
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")

d:\to_github\yandex_practicum_ds\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    max_features=100_000,
    stop_words="english",
    sublinear_tf=True,
    norm="l2"
)

### Константы

In [3]:
CV = 5
N_OPTUNA = 20
TEST_SIZE = 0.2
RANDOM_STATE = 20

In [4]:
# data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0])
df = pd.read_csv('./data/toxic_comments.csv', index_col=[0])
display(df.head())

,text,toxic
0,Explanation\r\nWhy the edits made under my use...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\r\nMore\r\nI can't make any real suggestions...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Подготовка текста

In [ ]:
def tfidf_clean(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\b[a-z]\b", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["text_clean"] = df["text"].progress_apply(tfidf_clean)

100%|██████████| 159292/159292 [00:03<00:00, 50454.81it/s]


In [6]:
display(df.sample(5))

,text,toxic,text_clean
62940,"Oh my... you're right, it looks like a dog's b...",0,oh my you re right it looks like dog breakfast...
115644,That’s not what A1 says: Please read WP:A1. It...,0,that not what says please read wp it says noth...
83961,"""\r\nThe edit was not exactly a revert - Books...",0,the edit was not exactly revert books commente...
18762,"""\r\n Oops, I was on the same track but going ...",0,oops was on the same track but going in the op...
143918,"""\r\n\r\n Name \r\n\r\nPaper Wrapped Cake? Th...",0,name paper wrapped cake that is lame that not ...


In [7]:
df_train, df_test = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df["toxic"]
)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

logger.info(f"Train size: {len(df_train)}, Test size: {len(df_test)}")

2026-01-08 11:06:21 [INFO] __main__: Train size: 127433, Test size: 31859


In [8]:
X_train = tfidf.fit_transform(df_train["text_clean"])
y_train = df_train["toxic"].values

X_test = tfidf.transform(df_test["text_clean"])
y_test = df_test["toxic"].values

logger.info(f"TF-IDF train shape: {X_train.shape}")
logger.info(f"TF-IDF test shape: {X_test.shape}")

2026-01-08 11:06:31 [INFO] __main__: TF-IDF train shape: (127433, 100000)
2026-01-08 11:06:31 [INFO] __main__: TF-IDF test shape: (31859, 100000)


## Обучение

In [9]:
# Проверяем баланс классов
logger.info(f"Распределение классов в train: {np.bincount(y_train)}")
logger.info(f"Доля positive класса: {y_train.mean():.3f}")

2026-01-08 11:06:31 [INFO] __main__: Распределение классов в train: [114484  12949]
2026-01-08 11:06:31 [INFO] __main__: Доля positive класса: 0.102


In [10]:
best_models = {}
models_to_run = ["LogisticRegression", "RidgeClassifier"]

for model_name in models_to_run:

    logger.info(f"===== START MODEL: {model_name} =====")

    def objective(trial):

        if model_name == "LogisticRegression":
            clf = LogisticRegression(
                C=trial.suggest_float("C", 1e-3, 10.0, log=True),
                max_iter=2000,
                class_weight="balanced",
                n_jobs=-1,
                solver="lbfgs"
            )

        elif model_name == "RidgeClassifier":
            clf = RidgeClassifier(
                alpha=trial.suggest_float("alpha", 1e-3, 10.0, log=True),
                class_weight="balanced"
            )

        cv = StratifiedKFold(
            n_splits=CV,
            shuffle=True,
            random_state=RANDOM_STATE
        )

        f1_scores = []

        for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
            X_tr, X_val = X_train[train_idx], X_train[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            clf.fit(X_tr, y_tr)
            y_pred = clf.predict(X_val)

            f1 = f1_score(y_val, y_pred)
            f1_scores.append(f1)

            trial.report(np.mean(f1_scores), step=fold_idx)
            if trial.should_prune():
                raise optuna.TrialPruned()

        return float(np.mean(f1_scores))

    study = optuna.create_study(
        direction="maximize",
        sampler=TPESampler(seed=RANDOM_STATE),
        pruner=MedianPruner(
            n_startup_trials=2,
            n_warmup_steps=1,
            interval_steps=1
        )
    )

    study.optimize(objective, n_trials=N_OPTUNA, show_progress_bar=False)

    best_models[model_name] = {
        "best_trial": study.best_trial,
        "best_value": study.best_value
    }

    logger.info(f"[{model_name}] COMPLETED | Best F1={study.best_value:.4f}")

# ===== выбор лучшей модели =====
best_model_name = max(best_models, key=lambda k: best_models[k]["best_value"])
best_trial = best_models[best_model_name]["best_trial"]
params = best_trial.params

if best_model_name == "LogisticRegression":
    final_model = LogisticRegression(
        **params,
        max_iter=2000,
        class_weight="balanced",
        n_jobs=-1
    )
else:
    final_model = RidgeClassifier(
        **params,
        class_weight="balanced"
    )

final_model.fit(X_train, y_train)
y_pred_test = final_model.predict(X_test)

test_f1 = f1_score(y_test, y_pred_test)

logger.info(
    f"Лучшая модель: {best_model_name} | "
    f"F1 на тесте={test_f1:.4f} | "
    f"Параметры={params}"
)

2026-01-08 11:06:31 [INFO] __main__: ===== START MODEL: LogisticRegression =====
2026-01-08 11:08:04 [INFO] __main__: [LogisticRegression] COMPLETED | Best F1=0.7749
2026-01-08 11:08:04 [INFO] __main__: ===== START MODEL: RidgeClassifier =====
2026-01-08 11:08:59 [INFO] __main__: [RidgeClassifier] COMPLETED | Best F1=0.7318
2026-01-08 11:09:00 [INFO] __main__: Лучшая модель: LogisticRegression | F1 на тесте=0.7684 | Параметры={'C': 9.665520684760546}


## Выводы

### Выводы по результатам эксперимента

#### Выполненные шаги

1. **Предобработка текста**
   - Приведение текста к нижнему регистру.
   - Удаление ссылок, спецсимволов, одиночных букв.
   - Очистка пробелов и стоп-слов.

2. **Преобразование текста в числовой формат**
   - Построение TF-IDF признаков с учётом униграмм и биграмм.
   - Ограничение по минимальной и максимальной частоте слов.
   - Нормализация L2 и сублинейная TF.

3. **Разделение выборки**
   - Train / Test split с стратификацией по целевому классу.
   - Сброс индексов для удобства работы.

4. **Подбор модели**
   - Оптимизация гиперпараметров LogisticRegression и RidgeClassifier через Optuna с кросс-валидацией.
   - Обучение на всей тренировочной выборке после подбора лучших параметров.

5. **Оценка результатов**
   - F1-score на тестовой выборке.
   - Проверка баланса классов.
   - Контроль утечек данных и согласованности действий.

#### Результаты

1. **Качество моделей**
- `LogisticRegression + TF-IDF` показала лучший результат:
  - CV F1 - 0.775
  - Test F1 - 0.768
- `RidgeClassifier` слабее:
  - CV F1 - 0.732  

2. **Обобщающая способность**
- Разница между CV и тестом минимальна переобучения нет.
- TF-IDF обучался только на train утечек данных нет.

3. **Оптимальность архитектуры**
- Для токсик-классификации TF-IDF + линейная модель достаточно:
  - захватывает лексику и n-граммы
  - нелинейность не критична

4. **LGBM / XGBoost**
- Эти модели могут дать чуть более высокий F1,  
  но обучение дольше и требует больше памяти.
- Прирост качества не оправдывает дополнительные ресурсы.

5. **Практический итог**
- Простые модели:
  - быстро обучаются
  - легко деплоятся
  - проще интерпретируются
  - стабильны в продакшене
- Качество уже близко к «потолку» для классического NLP без нейросетей.

**Итог:** проделанные действия позволили перейти с BERT на TF-IDF + LogisticRegression, что дало высокое качество, ускорило обучение, упростило код и инфраструктуру, устранив лишние вычислительные затраты.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны